In [1]:
import nibabel as nib
import numpy as np
import pandas as pd
from nilearn.image import resample_to_img
from nilearn import plotting
import os


In [2]:
# ============
# paths
# ============
atlas_path = "/home/lautaro/Documentos/BrainAgeNeXt/MedNeXt/BrainAgeNeXt/LRP_brainage_explainability/hammers_atlas/Hammers_mith-n30r95-MaxProbMap-full-MNI152-SPM12.nii.gz" #Atlas path
labels_path = "/home/lautaro/Documentos/BrainAgeNeXt/MedNeXt/BrainAgeNeXt/LRP_brainage_explainability/hammers_atlas/labels_Hammers.csv" #Atlas labels path
lrp_path = "/home/lautaro/Documentos/BrainAgeNeXt/MedNeXt/BrainAgeNeXt/LRP_brainage_explainability/lrp_mean_CP.nii.gz"   #LRP image path

In [ ]:
# -------------------
# Load Images
# -------------------
lrp_img = nib.load(lrp_path)
atlas_img = nib.load(atlas_path)
labels= pd.read_csv(labels_path)["structure"].tolist()

# Resample Atlas to LRP's map space
atlas_resampled = resample_to_img(atlas_img, lrp_img, interpolation="nearest")

#save resampled atlas if needed
#nib.save(atlas_resampled, "atlas_resampled.nii.gz")

if len(atlas_img.shape) == 4 and atlas_img.shape[-1] == 1:
    atlas_data_3d = np.squeeze(atlas_img.get_fdata(), axis=-1)
    atlas_img = nib.Nifti1Image(atlas_data_3d, atlas_img.affine, atlas_img.header)

# extract data arrays
lrp_data = lrp_img.get_fdata()
atlas_data = atlas_resampled.get_fdata()

# delete last dimension if it's singleton
if atlas_data.ndim == 4 and atlas_data.shape[-1] == 1:
    atlas_data = np.squeeze(atlas_data, axis=-1)


# -------------------
# compute explainability scores per region
# -------------------
region_scores = {}
for roi_val, roi_name in enumerate(labels, start=1):
    mask = atlas_data == roi_val
    roi_values = lrp_data[mask]
    if roi_values.size > 0:
        region_scores[roi_name] = np.mean(roi_values) 
    else:
        region_scores[roi_name] = np.nan

df_scores = pd.DataFrame.from_dict(region_scores, orient="index", columns=["explainability"])

/tmp/ipykernel_1049883/454746409.py:9: FutureWarning: 'force_resample' will be set to 'True' by default in Nilearn 0.13.0.
Use 'force_resample=True' to suppress this warning.
  atlas_resampled = resample_to_img(atlas_img, lrp_img, interpolation="nearest")
/tmp/ipykernel_1049883/454746409.py:9: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  atlas_resampled = resample_to_img(atlas_img, lrp_img, interpolation="nearest")
/tmp/ipykernel_1049883/454746409.py:9: UserWarning: The provided image has no sform in its header. Please check the provided file. Results may not be as expected.
  atlas_resampled = resample_to_img(atlas_img, lrp_img, interpolation="nearest")


In [ ]:
#df_scores.to_csv("lrp_region_scores_UNSAM.csv")  # save results in a csv

In [ ]:
#read al csv files previously saved
adniscores=pd.read_csv("lrp_region_scores_ADNICN.csv")
ncnscores=pd.read_csv("lrp_region_scores_JUK.csv")
cogconvarscores=pd.read_csv("lrp_region_scores_RRIB.csv")
cpscores=pd.read_csv("lrp_region_scores_UNSAM.csv")

In [ ]:
#normalize intensities by its maximum voxel's relevance score
adniscores['explainability']=adniscores['explainability']/13775.78
ncnscores['explainability']=ncnscores['explainability']/11977.31
cogconvarscores['explainability']=cogconvarscores['explainability']/13775.78
cpscores['explainability']=cpscores['explainability']/15551.63


In [5]:
adniscores

,Region,Explicabilidad
0,cerebellum-R,0.343240
1,caudate-nucleus-L,0.291770
2,Lateral-ventricle-excluding-temporal-horn-R,0.285144
3,Lateral-ventricle-excluding-temporal-horn-L,0.280530
4,brainstem-excluding-substantia-nigra,0.264902
...,...,...
90,PL-angular-gyrus-R,-0.151361
91,FL-lateral-orbital-gyrus-R,-0.158931
92,FL-lateral-orbital-gyrus-L,-0.166258
93,PL-angular-gyrus-L,-0.181537


In [ ]:
# summatory of explainability scores
final_scores = (
    adniscores.set_index("Region")["explainability"] +
    cogconvarscores.set_index("Region")["explainability"] +
    cpscores.set_index("Region")["explainability"] +
    ncnscores.set_index("Region")["explainability"]
).reset_index()

final_scores.columns = ["Region", "Total explainability"]

final_scores = final_scores.sort_values(by="Total explainability", ascending=False)
final_scores

,Region,Total Explicabilidad
29,Lateral-ventricle-excluding-temporal-horn-R,1.543994
28,Lateral-ventricle-excluding-temporal-horn-L,1.134872
94,thalamus-R,1.053069
72,corpus-callosum,1.000259
68,caudate-nucleus-L,0.931232
...,...,...
62,TL-superior-temporal-gyrus-anterior-part-L,-0.404485
39,PL-angular-gyrus-R,-0.414117
44,PL-supramarginal-gyrus-L,-0.421193
8,FL-lateral-orbital-gyrus-L,-0.447978


In [ ]:
#final_scores.to_csv("lrp_region_scores_total.csv", index=False)